In [20]:
# load our dependencies
import time
import numpy as np
import warnings
import matplotlib.pyplot as plt

from operator import itemgetter
from hyperopt import hp, fmin, rand, tpe, space_eval, Trials, STATUS_OK
from scipy.stats import randint, uniform, lognorm
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.datasets import load_boston
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.cross_validation import train_test_split, cross_val_score

%matplotlib inline
plt.style.use("ggplot")
warnings.filterwarnings("ignore")

## Optimizing Hyper-parameters for Machine Learning (part 2)

part 1 https://github.com/Eric-Xu/nycdssg_talks/blob/master/08_03_2015_hyperparams/grid_and_random_search_demo.ipynb

**Objective:** Compare hyper-parameter results found through grid search, random search, and hyperopt search in predicting Boston housing prices.

In [21]:
# read in the Boston housing dataset
# description of dataset can be found at:
# https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/datasets/descr/boston_house_prices.rst
boston = load_boston()

# set our features and target
X, y = boston.data, boston.target

# peek at our dataset
print "Total rows:", X.shape[0]
print "Feature names:", boston["feature_names"]
print "First row features:", X[0]
print "First row target:", y[0]

Total rows: 506
Feature names: ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT' 'MEDV']
First row features: [  6.32000000e-03   1.80000000e+01   2.31000000e+00   0.00000000e+00
   5.38000000e-01   6.57500000e+00   6.52000000e+01   4.09000000e+00
   1.00000000e+00   2.96000000e+02   1.53000000e+01   3.96900000e+02
   4.98000000e+00]
First row target: 24.0


### MSE of default classifier

In [34]:
# initialize a gradient boosting classifier with default parameters
default_clf = GradientBoostingRegressor()

# inspect the default parameters
print default_clf

# run a 10-fold cross-validation with our classifier
mse_scores = cross_val_score(default_clf, X, y, cv=10, scoring="mean_squared_error")

# our goal is to find the parameter configuration that minimizes the mean squared error
print "MSE of default classifier", np.mean(-mse_scores)

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss=ls,
             max_depth=3, max_features=None, min_samples_leaf=1,
             min_samples_split=2, n_estimators=100, random_state=None,
             subsample=1.0, verbose=0)
baseline MSE 18.4607094065


### Lowest MSE of best parameters found by `GridSearchCV`

In [62]:
# helper function to get best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1))[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean MSE score: {0:.3f}".format(score.mean_validation_score))
        print("Parameters: {0}\n".format(score.parameters))

In [59]:
%%time

# define the parameter search space
param_grid = {
    "n_estimators": [100,500,1000],
    "learning_rate": [0.01,0.1,1],
    "max_depth": [1,5,10],
    "min_samples_split": [1,5,10],
    "loss": ["ls", "lad"]
}

# create a MSE scorer object
mse_scorer = make_scorer(mean_squared_error)

# instantiate our grid search running a 10-fold cross-validation for each parameter combination
grid_clf = GradientBoostingRegressor()
grid = GridSearchCV(grid_clf, param_grid, cv=10, scoring=mse_scorer, n_jobs=-1)

# fit grid search with data
grid.fit(X, y)

report(grid.grid_scores_)
print "Total {0} parameter combinations tested".format(len(grid.grid_scores_))

Model with rank: 1
Mean MSE score: 19.782
Parameters: {'min_samples_split': 1, 'loss': 'lad', 'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 5}

2 trials sampled
CPU times: user 81.9 ms, sys: 23.5 ms, total: 105 ms
Wall time: 6.77 s


### Lowest MSE of best parameters found by `RandomizedSearchCV`

In [64]:
%%time

# define the parameter search space
param_dist = {
    "n_estimators": randint(1,1000),
    "learning_rate": lognorm(0.5,0,1),
    "max_depth": randint(1, 10),
    "min_samples_split": randint(1, 10),
    "loss": ["ls", "lad"]
}

# create a MSE scorer object
mse_scorer = make_scorer(mean_squared_error)

# instantiate our random search running a 10-fold cross-validation for each trial
n_trials = 60
rand_clf = GradientBoostingRegressor()
rand = RandomizedSearchCV(rand_clf, param_dist, cv=10, scoring=mse_scorer, n_iter=n_trials, random_state=99, n_jobs=-1)

# fit grid search with data
rand.fit(X, y)

report(rand.grid_scores_)
print "Total {0} trials sampled".format(n_trials)

Model with rank: 1
Mean MSE score: 23.495
Parameters: {'min_samples_split': 3, 'loss': 'ls', 'learning_rate': 0.4464848850550619, 'n_estimators': 464, 'max_depth': 5}

Model with rank: 2
Mean MSE score: 23.994
Parameters: {'min_samples_split': 1, 'loss': 'ls', 'learning_rate': 0.82917849749949668, 'n_estimators': 693, 'max_depth': 1}

Model with rank: 3
Mean MSE score: 24.430
Parameters: {'min_samples_split': 8, 'loss': 'lad', 'learning_rate': 0.65224870034040483, 'n_estimators': 131, 'max_depth': 5}

Total 10 trials sampled
CPU times: user 2.83 s, sys: 57.4 ms, total: 2.88 s
Wall time: 48.7 s
